# Hello World Kubeflow Pipeline

In [ ]:
from datetime import datetime
from pathlib import Path

from google.cloud import aiplatform
from google.oauth2 import service_account
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline

## Config

In [ ]:
# GCP related config
PIPELINE_NAME = "hello-world-pipeline-pytorch"
BUCKET_NAME = "my-vertex-playground-bucket"  # set up beforehand
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/{PIPELINE_NAME}/"

PROJECT_ID = "open-problems-multimodal"  # set up beforehand
REGION = "us-central1"
SERVICE_ACCOUNT = (
    "vertex@open-problems-multimodal.iam.gserviceaccount.com"  # set up beforehand
)
GCP_CREDENTIALS_PATH = Path().home() / "keys" / "my-vertex-playground-key.json"

# Local path to the compiled pipeline json file
PIPELINE_JSON_PATH = "compiled_pipeline.json"

## Components

In [ ]:
@dsl.component(
    base_image="gcr.io/deeplearning-platform-release/pytorch-gpu.1-13:latest"
)
def main():
    import torch

    print(torch.__version__)
    print(torch.cuda.is_available())
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.device_count())
    print(torch.cuda.current_device())



## Pipeline

In [ ]:
@pipeline(
    name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
)
def my_pipeline():
    main_task = main()
    main_task.set_gpu_limit(1)
    main_task.add_node_selector_constraint(label_name='cloud.google.com/gke-accelerator', value='nvidia-tesla-t4')


# Compile the pipeline
compiler.Compiler().compile(pipeline_func=my_pipeline, package_path=PIPELINE_JSON_PATH)

## Submit job

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d-%H%M%S")
JOB_NAME = f"{PIPELINE_NAME}-{TIMESTAMP}"
credentials = service_account.Credentials.from_service_account_file(
    GCP_CREDENTIALS_PATH
)
job = aiplatform.PipelineJob(
    display_name=JOB_NAME,
    credentials=credentials,
    template_path=PIPELINE_JSON_PATH,
    job_id=JOB_NAME,
    pipeline_root=PIPELINE_ROOT,
    enable_caching=True,
    project=PROJECT_ID,
    location=REGION,
)
job.submit(service_account=SERVICE_ACCOUNT)